## Part 2: Embedding Julia into Python

We can use PythonCall for integrating Python’s vast ecosystem into Julia projects and JuliaCall for embedding high-performance Julia code into Python scripts.

You’ll see how easy it is to blend these languages and why it’s worth the effort.

In [1]:
from juliacall import Main as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
%load_ext juliacall

In [3]:
%julia using Pkg

In [4]:
%julia Pkg.add("UnROOT")

   Resolving package versions...
  No Changes to `~/anaconda3/envs/julia_hep_2024/julia_env/Project.toml`
  No Changes to `~/anaconda3/envs/julia_hep_2024/julia_env/Manifest.toml`


In [5]:
%julia using UnROOT

In [6]:
file = jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

In [7]:
%%timeit
jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

907 μs ± 14.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
file

ROOTFile with 1 entry and 18 streamers.
./data/SMHiggsToZZTo4L.root
└─ Events (TTree)
   ├─ "run"
   ├─ "luminosityBlock"
   ├─ "event"
   ├─ "⋮"
   ├─ "Electron_dzErr"
   ├─ "MET_pt"
   └─ "MET_phi"


In [9]:
events = jl.Main.LazyTree(file, "Events")

In [10]:
%%timeit
jl.Main.LazyTree(file, "Events")

293 μs ± 14.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
jl.include('awkward_analyzer_functions.jl')

invariant_mass (generic function with 1 method)

```julia
using AwkwardArray

function make_record_array(events)
    array = AwkwardArray.RecordArray(
        NamedTuple{(:pt, :eta, :phi, :mass, :charge, :isolation)}((
            AwkwardArray.from_iter(events.Muon_pt),
            AwkwardArray.from_iter(events.Muon_eta), 
            AwkwardArray.from_iter(events.Muon_phi), 
            AwkwardArray.from_iter(events.Muon_mass), 
            AwkwardArray.from_iter(events.Muon_charge), 
            AwkwardArray.from_iter(events.Muon_pfRelIso03_all),
        )
    ))
    return AwkwardArray.convert(array)
end
```

In [12]:
muons = jl.make_record_array(events)

In [13]:
%%time
jl.make_record_array(events)

CPU times: user 203 ms, sys: 15.8 ms, total: 219 ms
Wall time: 218 ms


<Array [{pt: [63, ...], eta: [...], ...}, ...] type='299973 * {pt: var * fl...'>

In [14]:
muons.show(type=True)

type: 299973 * {
    pt: var * float32,
    eta: var * float32,
    phi: var * float32,
    mass: var * float32,
    charge: var * int32,
    isolation: var * float32
}
[{pt: [63, 38.1, 4.05], eta: [-0.719, ..., -0.321], phi: [...], mass: ..., ...},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 {pt: [54.3, 23.5, ..., 8.39, 3.49], eta: [-1.06, ...], phi: [...], ...},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 {pt: [38.5, 47], eta: [0.315, -0.119], phi: [2.05, ...], mass: [...], ...},
 {pt: [4.45], eta: [-0.986], phi: [1.12], mass: [0.106], charge: [1], ...},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 {pt: [], eta: [], phi: [], mass: [], charge: [], isolation: []},
 ...,
 {pt: [37.2, 50.1], eta: [1.1, 0.412], phi: [-0.875, ...], mass: [...], ...},
 {pt: [43.2, 24], eta: [2.15, 0.421], phi: 

In [15]:
import awkward as ak

In [16]:
muons = ak.zip({
                "pt": muons.pt,
                "eta": muons.eta,
                "phi": muons.phi,
                "mass": muons.mass,
                "charge": muons.charge,
                "isolation": muons.isolation,
            },
            with_name="PtEtaPhiMCandidate",)

In [17]:
cutflow = dict()

# Sort muons by transverse momentum
muons = muons[ak.argsort(muons.pt, axis=1)]

cutflow["all events"] = ak.num(muons, axis=0)

# Quality and minimum pt cuts
muons = muons[(muons.pt > 5) & (muons.isolation < 0.2)]
cutflow["at least 4 good muons"] = ak.sum(ak.num(muons) >= 4)

In [18]:
# reduce first axis: skip events without enough muons
muons = muons[ak.num(muons) >= 4]

```julia
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.ListArray{
        AwkwardArray.Index64,
        AwkwardArray.PrimitiveArray{Int64},
    },
}()

function find_4lep(events_leptons)

    for leptons in events_leptons
        nlep = length(leptons)
        for i0 in 1:nlep
            for i1 in (i0 + 1):nlep
                if leptons[i0][:charge] + leptons[i1][:charge] != 0
                    continue
                end
                for i2 in 1:nlep
                    for i3 in (i2 + 1):nlep
                        if length(Set([i0, i1, i2, i3])) < 4
                            continue
                        end
                        if leptons[i2][:charge] + leptons[i3][:charge] != 0
                            continue
                        end
                        
                        push!(four_muons.content.content, (i0 - 1)) # Julia is 1-based, subtract 1 for 0-based indexing
                        push!(four_muons.content.content, (i1 - 1))
                        push!(four_muons.content.content, (i2 - 1))
                        push!(four_muons.content.content, (i3 - 1))
                        AwkwardArray.end_list!(four_muons.content)
                    end
                end
            end 
        end
        AwkwardArray.end_list!(four_muons)
    end
    return four_muons
end

```

In [19]:
good_four_muons = jl.find_4lep(muons[1:10])

In [20]:
good_four_muons = jl.find_4lep(muons)

In [21]:
%%time
jl.find_4lep(muons)

CPU times: user 274 ms, sys: 7.73 ms, total: 282 ms
Wall time: 282 ms


29917-element AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int64, Vector{Int64}, :default}, :default}, :default}:
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2, 3, 0, 1]]
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]]
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]]
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 2, 1, 4], [0, 2, 3, 4], [0, 4, 1, 2], ..., [3, 4, 0, 2], [3, 4, 1, 2]]
 0-element AwkwardArray.ListArray{Vector{Int64}, AwkwardArray.PrimitiveArray{Int64, Vector{Int64}, :default}, :default}
 ⋮
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]]
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2, 3, 0, 1]]
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1,

In [22]:
fourmuon = jl.AwkwardArray.convert(good_four_muons)

In [23]:
fourmuon.show(type=True)

type: 29917 * var * var * int64
[[[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2, 3, 0, 1]],
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]],
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]],
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 2, 1, 4], [0, 2, 3, 4], [0, 4, ..., 2], ..., [3, 4, 0, 2], [3, 4, 1, 2]],
 [],
 ...,
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2, 3, 0, 1]],
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]],
 [[0, 2, 1, 3], [0, 3, 1, 2], [1, 2, 0, 3], [1, 3, 0, 2]],
 [[0, 1, 2, 3], [0, 3, 1, 2], [1, 2, 0, 3], [2, 3, 0, 1]],
 [],
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2, 3, 0, 1]],
 [[0, 1, 2, 3], [0, 2, 1, 3], [1, 3, 0, 2], [2

Let's go to the next [notebook](AwkwardArray_Julia_Python-part3.ipynb).